# Coronavirus Tracker 

### Disclaimer
##### The following datasets have been recorded from: 
 * https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data
 * https://www.worldometers.info/coronavirus/

These datasets are updated on a daily basis by the Johns Hopkins Univerity

In [176]:
from __future__ import print_function
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px
from collections import namedtuple
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")
import pathlib
from datetime import date,timedelta
import os
import io

In [178]:
#Reading JHU database
url1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
df_r = pd.read_csv(url1, index_col=0)

url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df_c = pd.read_csv(url2, index_col=0)

url3 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df_d = pd.read_csv(url3, index_col=0)


In [179]:
df_r.reset_index(inplace=True)
df_c.reset_index(inplace=True)
df_d.reset_index(inplace=True)

In [180]:
#Renaming cols
df_c.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
df_r.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
df_d.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
#Renaming country names
df_r['country'] = np.where(df_r.country == 'Korea, South','South Korea',df_r['country'])
df_d['country'] = np.where(df_d.country == 'Korea, South','South Korea',df_d['country'])
df_c['country'] = np.where(df_c.country == 'Korea, South','South Korea',df_c['country'])
#Dropping diamond pricess and other cruises
df_r = df_r[~(df_r.country.isin(["Diamond Princess","MS Zaandam"]))]
df_d = df_d[~(df_d.country.isin(["Diamond Princess","MS Zaandam"]))]
df_c = df_c[~(df_c.country.isin(["Diamond Princess","MS Zaandam"]))]

In [181]:
df_c_melt=df_c_ge.copy()
df_d_melt=df_d_ge.copy()
df_r_melt=df_r_ge.copy()

In [183]:
df_c_ge.drop(columns=['province_state','Lat','Long'],inplace=True,axis=1)
df_c_ge.rename(columns={df_c_ge.columns[-1]:"current_day_confirmed",df_c_ge.columns[-2]: "prev_day_confirmed" }, inplace = True)
df_c_ge['prev_day_diff_confrimed'] = df_c_ge['current_day_confirmed'] - df_c_ge['prev_day_confirmed']
df_c_1 = df_c_ge[['country','prev_day_confirmed','current_day_confirmed','prev_day_diff_confrimed']]
df_c_grpd = df_c_1.groupby('country').sum()

df_r_ge.drop(columns=['province_state','Lat','Long'],inplace=True,axis=1)
df_r_ge.rename(columns={df_r_ge.columns[-1]:"current_day_recover",df_r_ge.columns[-2]: "prev_day_recover" }, inplace = True)
df_r_ge['prev_day_diff_recover'] = df_r_ge['current_day_recover'] - df_r_ge['prev_day_recover']
df_r_1 = df_r_ge[['country','prev_day_recover','current_day_recover','prev_day_diff_recover']]
df_r_grpd = df_r_1.groupby('country').sum()

df_d_ge.drop(columns=['province_state','Lat','Long'],inplace=True,axis=1)
df_d_ge.rename(columns={df_d_ge.columns[-1]:"current_day_death",df_d_ge.columns[-2]: "prev_day_death" }, inplace = True)
df_d_ge['prev_day_diff_death'] = df_d_ge['current_day_death'] - df_d_ge['prev_day_death']
df_d_1 = df_d_ge[['country','prev_day_death','current_day_death','prev_day_diff_death']]
df_d_grpd = df_d_1.groupby('country').sum()

In [184]:
df_merged1 = pd.concat([df_c_grpd, df_r_grpd], axis=1)
df_merged_global = pd.concat([df_merged1,df_d_grpd],axis=1)
df_merged_global.reset_index(inplace=True)

In [185]:
#create the region column as well
who_region = pd.read_csv('who_region.csv')
who_region.rename(columns={'Country/Region':'country','WHO Region':'region'},inplace= True)
df_merged_global_who_region = df_merged_global[['country','prev_day_confirmed','current_day_confirmed','prev_day_diff_confrimed','prev_day_recover',
         'current_day_recover','prev_day_diff_recover','prev_day_death','current_day_death','prev_day_diff_death']].merge(who_region[['country','region']],on='country',how='left')


In [187]:
df_confirmed = df_c_melt.melt(id_vars=['country','province_state','Lat','Long'],
                    var_name='Date',
                    value_name='total_confrimed')
df_recover = df_r_melt.melt(id_vars=['country','province_state','Lat','Long'],
                    var_name='Date',
                    value_name='total_recover')
df_deaths = df_d_melt.melt(id_vars=['country','province_state','Lat','Long'],
                    var_name='Date',
                    value_name='total_deaths')

In [188]:
df_confirmed.head()

,country,province_state,Lat,Long,Date,total_confrimed
0,Albania,NaN,41.1533,20.1683,1/22/20,0
1,Algeria,NaN,28.0339,1.6596,1/22/20,0
2,Angola,NaN,-11.2027,17.8739,1/22/20,0
3,Argentina,NaN,-38.4161,-63.6167,1/22/20,0
4,Australia,Australian Capital Territory,-35.4735,149.0124,1/22/20,0


In [189]:

df_confirmed['key'] = df_confirmed['province_state'].astype(str)+'-'+df_confirmed['country'].astype(str)+'-'+df_confirmed['Date'].astype(str)

df_recover['key'] = df_recover['province_state'].astype(str) + '-'+df_recover['country'].astype(str)+'-'+df_recover['Date'].astype(str)

df_deaths['key'] = df_deaths['province_state'].astype(str) + '-'+ df_deaths['country'].astype(str) + '-' + df_deaths['Date'].astype(str)



In [190]:
df_1= df_confirmed[['province_state','country','Lat','Long','Date','total_confrimed','key']].merge(df_deaths[['total_deaths','key']], on='key', how='left')
df_2 = df_1[['province_state','country','Lat','Long','Date','total_confrimed','total_deaths','key']].merge(df_recover[['total_recover','key']],on='key',how='left')
df_global = df_2.drop(columns=['key'])


In [191]:
df_global.head()

,province_state,country,Lat,Long,Date,total_confrimed,total_deaths,total_recover
0,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0.0
1,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0.0
2,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0.0
3,NaN,Argentina,-38.4161,-63.6167,1/22/20,0,0,0.0
4,Australian Capital Territory,Australia,-35.4735,149.0124,1/22/20,0,0,0.0


In [192]:
df_global.total_confrimed.fillna(0,inplace=True)
df_global.total_deaths.fillna(0,inplace=True)
df_global.total_recover.fillna(0,inplace=True)

In [193]:
#create the region column as well
df_global = df_global[['province_state','country','Lat','Long','Date','total_confrimed','total_deaths','total_recover']].merge(who_region[['country','region']],on='country',how='left')
df_global.drop(['Lat','Long'],inplace=True,axis=1)

In [194]:
df_global.to_excel(r'raw_global_data.xlsx')

In [195]:
df_global.to_excel(r'raw_global_data_for_graphs.xlsx')

# India Analysis - State Level

### Disclaimer
##### The following datasets have been recorded from: 
 * https://github.com/kalyaniuniversity/COVID-19-Datasets
 * https://www.worldometers.info/coronavirus/
 * https://www.mohfw.gov.in/

These datasets are updated on a daily basis by the Kalyani Univerity and the Indian Govt

In [198]:
url4 = 'https://raw.githubusercontent.com/kalyaniuniversity/COVID-19-Datasets/master/India%20Statewise%20Recovery%20Cases/COVID19_INDIA_STATEWISE_TIME_SERIES_RECOVERY.csv'
df_rec_india = pd.read_csv(url4, index_col=0)

url5 = 'https://raw.githubusercontent.com/kalyaniuniversity/COVID-19-Datasets/master/India%20Statewise%20Confirmed%20Cases/COVID19_INDIA_STATEWISE_TIME_SERIES_CONFIRMED.csv'
df_con_india = pd.read_csv(url5, index_col=0)

url6 = 'https://raw.githubusercontent.com/kalyaniuniversity/COVID-19-Datasets/master/India%20Statewise%20Death%20Cases/COVID19_INDIA_STATEWISE_TIME_SERIES_DEATH.csv'

df_dea_india = pd.read_csv(url6, index_col=0)

df_rec_india.reset_index(inplace=True)
df_con_india.reset_index(inplace=True)
df_dea_india.reset_index(inplace=True)

In [199]:
df_c_India_melt=df_con_india.copy()
df_d_India_melt=df_dea_india.copy()
df_r_India_melt=df_rec_india.copy()


In [200]:
#confirmed
df_con_india.drop(columns=['CODE','LATITUDE','LONGITUDE','PER CAPITA INCOME (INR)','AVERAGE TEMPERATURE (°C)'],inplace=True,axis=1)
df_con_india.rename(columns={'STATE/UT':'STATE_UT',df_con_india.columns[-1]:"current_day_confirmed",df_con_india.columns[-2]: "prev_day_confirmed" }, inplace = True)
df_con_india['prev_day_diff_confrimed'] = df_con_india['current_day_confirmed'] - df_con_india['prev_day_confirmed']
df_con_india1 = df_con_india[['STATE_UT','POPULATION','prev_day_confirmed','current_day_confirmed','prev_day_diff_confrimed']]
#Recover
df_rec_india.drop(columns=['CODE','LATITUDE','LONGITUDE','PER CAPITA INCOME (INR)','AVERAGE TEMPERATURE (°C)'],inplace=True,axis=1)
df_rec_india.rename(columns={'STATE/UT':'STATE_UT',df_rec_india.columns[-1]:"current_day_recover",df_rec_india.columns[-2]: "prev_day_recover" }, inplace = True)
df_rec_india['prev_day_diff_recover'] = df_rec_india['current_day_recover'] - df_rec_india['prev_day_recover']
df_rec_india1 = df_rec_india[['STATE_UT','prev_day_recover','current_day_recover','prev_day_diff_recover']]
#Death
df_dea_india.drop(columns=['CODE','LATITUDE','LONGITUDE','PER CAPITA INCOME (INR) ','AVERAGE TEMPERATURE (°C)'],inplace=True,axis=1)
df_dea_india.rename(columns={'STATE/UT':'STATE_UT',df_dea_india.columns[-1]:"current_day_deaths",df_dea_india.columns[-2]: "prev_day_deaths" }, inplace = True)
df_dea_india['prev_day_diff_deaths'] = df_dea_india['current_day_deaths'] - df_dea_india['prev_day_deaths']
df_dea_india1 = df_dea_india[['STATE_UT','prev_day_deaths','current_day_deaths','prev_day_diff_deaths']]

In [201]:
df_merged1_India = df_con_india1.merge(df_rec_india1,on='STATE_UT',how='left')
df_merged_India = df_merged1_India.merge(df_dea_india1,on='STATE_UT',how='left')
df_merged_India.rename(columns={'STATE_UT':'States_UT'},inplace=True)
df_merged_India = df_merged_India[~(df_merged_India.States_UT.isin(["Total","Unassigned State","Daman and Diu"]))]
# df_merged_India.set_index('STATE_UT',inplace=True)

In [202]:
df_confirmed_india = df_c_India_melt.melt(id_vars=['STATE/UT','CODE','LATITUDE','LONGITUDE','PER CAPITA INCOME (INR)','POPULATION','AVERAGE TEMPERATURE (°C)'],
                    var_name='Date',
                    value_name='total_confirmed')
df_deaths_india = df_d_India_melt.melt(id_vars=['STATE/UT','CODE','LATITUDE','LONGITUDE','PER CAPITA INCOME (INR) ','POPULATION','AVERAGE TEMPERATURE (°C)'],
                    var_name='Date',
                    value_name='total_deaths')
df_recover_india = df_r_India_melt.melt(id_vars=['STATE/UT','CODE','LATITUDE','LONGITUDE','PER CAPITA INCOME (INR)','POPULATION','AVERAGE TEMPERATURE (°C)'],
                    var_name='Date',
                    value_name='total_recovered')

In [203]:
col = ['AVERAGE TEMPERATURE (°C)','PER CAPITA INCOME (INR)','LATITUDE','LONGITUDE','POPULATION']
df_confirmed_india.drop(col,axis=1,inplace=True)
df_recover_india.drop(col,axis=1,inplace=True)
col1 = ['AVERAGE TEMPERATURE (°C)','PER CAPITA INCOME (INR) ','LATITUDE','LONGITUDE','POPULATION']
df_deaths_india.drop(col1,axis=1,inplace=True)

df_confirmed_india['key'] = df_confirmed_india['STATE/UT'].astype(str)+'-'+df_confirmed_india['CODE'].astype(str)+'-'+df_confirmed_india['Date'].astype(str)
df_recover_india['key'] = df_recover_india['STATE/UT'].astype(str) + '-'+df_recover_india['CODE'].astype(str)+'-'+df_recover_india['Date'].astype(str)
df_deaths_india['key'] = df_deaths_india['STATE/UT'].astype(str) + '-'+ df_deaths_india['CODE'].astype(str) + '-' + df_deaths_india['Date'].astype(str)

df_1_India= df_confirmed_india[['STATE/UT','CODE','Date','total_confirmed','key']].merge(df_deaths_india[['total_deaths','key']], on='key', how='left')
df_2_India = df_1_India[['STATE/UT','CODE','Date','total_confirmed','total_deaths','key']].merge(df_recover_india[['total_recovered','key']],on='key',how='left')
df_India = df_2_India.drop(columns=['key'])
df_India.rename(columns={'STATE/UT':'STATE_UT'},inplace=True)
df_India = df_India[~(df_India.STATE_UT.isin(["Total","Unassigned State","Daman and Diu"]))]
df_India.set_index('STATE_UT',inplace=True)
df_India.to_excel(r'raw_data_India.xlsx')

In [204]:
df_India.head()

,CODE,Date,total_confirmed,total_deaths,total_recovered
STATE_UT,,,,,
Andaman and Nicobar Islands,an,1/30/2020,0,0,0
Andhra Pradesh,ap,1/30/2020,0,0,0
Arunachal Pradesh,ar,1/30/2020,0,0,0
Assam,as,1/30/2020,0,0,0
Bihar,br,1/30/2020,0,0,0


# State level data from MOHFW

In [206]:
import requests, json
from pandas.io.json import json_normalize
url = 'https://www.mohfw.gov.in/data/datanew.json'
res = requests.get(url)
with open("data.json", "w") as f:
    json.dump(res.json(), f)
df_in = pd.read_json('data.json', orient='columns')
df_in.drop(['state_code','sno'],inplace=True, axis=1)
df_in['state_name'] = df_in['state_name'].replace({'Telengana': 'Telangana','Dadra and Nagar Haveli and Daman and Diu':'Dadra and Nagar Haveli'})
df_in = df_in[df_in.state_name != '']
df_in.rename(columns={'state_name':'States_UT','active': 'ActiveCases','cured':'Recovered','death':'Deaths',
                     'positive':'Confirmed'},inplace=True)
df_in["POPULATION"] = df_in["POPULATION"].str.replace(',', '').astype(float).astype(int)
df_in.drop('state',inplace=True,axis=1)
df_in.set_index('States_UT',inplace=True)
df_in.to_excel(r'India_state_data.xlsx')

# ******************************************** End of data prep for India ***********************************************

# USA Analysis - State Level

##### The following datasets have been recorded from: 
 * https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data
 * https://www.worldometers.info/coronavirus/

These datasets are updated on a daily basis by the Johns Hopkins Univerity

In [208]:
url8 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
df_c_US = pd.read_csv(url8, index_col=0)

url9 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_d_US = pd.read_csv(url9, index_col=0)


In [209]:
df_c_US.reset_index(inplace=True)
df_d_US.reset_index(inplace=True)

In [210]:
df_c_US = df_c_US[~(df_c_US.Province_State.isin(["Hawaii","Alaska","American Samoa", "Diamond Princess","Grand Princess","Guam","Northern Mariana Islands","Virgin Islands"]))]
df_d_US = df_d_US[~(df_d_US.Province_State.isin(["Hawaii","Alaska","American Samoa", "Diamond Princess","Grand Princess","Guam","Northern Mariana Islands","Virgin Islands"]))]
df_c_US_melt=df_c_US.copy()
df_d_US_melt=df_d_US.copy()

In [211]:
# df_d_US.drop(columns=['UID','iso2','iso3','code3','FIPS','Admin2','Country_Region','Combined_Key'],inplace=True,axis=1)
df_c_US.drop(columns=['UID','iso2','iso3','code3','FIPS','Admin2','Country_Region','Combined_Key','Lat','Long_'],inplace=True,axis=1)
df_c_US.rename(columns={df_c_US.columns[-1]:"current_day_confirmed",df_c_US.columns[-2]: "prev_day_confirmed" }, inplace = True)
df_c_US['prev_day_diff_confrimed'] = df_c_US['current_day_confirmed'] - df_c_US['prev_day_confirmed']
df_c_US_1 = df_c_US[['Province_State','prev_day_confirmed','current_day_confirmed','prev_day_diff_confrimed']]
df_c_US_grpd = df_c_US_1.groupby('Province_State').sum()
# df_d_US.drop(columns=['UID','iso2','iso3','code3','FIPS','Admin2','Country_Region','Combined_Key'],inplace=True,axis=1)
df_d_US.drop(columns=['UID','iso2','iso3','code3','FIPS','Admin2','Country_Region','Combined_Key','Lat','Long_'],inplace=True,axis=1)
df_d_US.rename(columns={df_d_US.columns[-1]:"current_day_death",df_d_US.columns[-2]: "prev_day_death" }, inplace = True)
df_d_US['prev_day_diff_death'] = df_d_US['current_day_death'] - df_d_US['prev_day_death']
df_d_US_1 = df_d_US[['Province_State','prev_day_death','current_day_death','prev_day_diff_death']]
df_d_US_grpd = df_d_US_1.groupby('Province_State').sum()
df_merged1_US = pd.concat([df_c_US_grpd, df_d_US_grpd], axis=1)
df_merged1_US.reset_index(inplace=True)

In [212]:
df_confirmed_US = df_c_US_melt.melt(id_vars=['UID','iso2','iso3','code3','FIPS','Admin2','Province_State','Country_Region','Lat','Long_','Combined_Key'],
                    var_name='Date',
                    value_name='total_confirmed')
df_deaths_US = df_d_US_melt.melt(id_vars=['UID','iso2','iso3','code3','FIPS','Admin2','Province_State','Country_Region','Lat','Long_','Combined_Key','Population'],
                    var_name='Date',
                    value_name='total_deaths')

In [213]:
col_US = ['UID','iso2','iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key']
col_US1 = ['UID','iso2','iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key','Population']
df_confirmed_US.drop(col_US,axis=1,inplace=True)
df_deaths_US.drop(col_US1,axis=1,inplace=True)
df_deaths_US = df_deaths_US[~(df_deaths_US.Province_State.isin(["Puerto Rico"]))]
df_confirmed_US = df_confirmed_US[~(df_confirmed_US.Province_State.isin(["Puerto Rico"]))]

In [214]:
df_deaths_US.set_index('Province_State',inplace=True)
df_confirmed_US.set_index('Province_State',inplace=True)

In [215]:
df_confirmed_US.to_excel(r'confirmed_US.xlsx')
df_deaths_US.to_excel(r'deaths_US.xlsx')

In [216]:
import requests
from bs4 import BeautifulSoup
url_us = 'https://www.worldometers.info/coronavirus/country/us/'
html_page=requests.get(url_us).text
soup = BeautifulSoup(html_page,'lxml')
get_table = soup.find("table",id="usa_table_countries_today")
get_table_data =get_table.tbody.find_all("tr")
dic={}
for i in range(len(get_table_data)):
    try:
        key = get_table_data[i].find_all("a",href=True)[0].string
    except:
        key = get_table_data[i].find_all("td")[0].string # there is not href inside the td text
    
    values = [j.string for j in get_table_data[i].find_all('td')]
    dic[key] =values
    
column_names_US = ["Total_Cases","New_Cases","Total_Deaths","New_Deaths","Total_Recovered","ActiveCases","TotCases1MPop","Deaths1MPop","TotalTests","Tests1MPop"]
US_state_data = pd.DataFrame(dic).iloc[1:,:].T.iloc[:,:10]
US_state_data.columns=column_names_US
US_state_data.reset_index(inplace = True)
US_state_data.rename(columns={'index': 'USA State'},inplace=True)
US_state_data = US_state_data.replace({'\n+':''}, regex=True)
US_state_data['New_Cases'] =US_state_data['New_Cases'].str.replace('+','') 
US_state_data['New_Deaths'] =US_state_data['New_Deaths'].str.replace('+','') 
US_state_data.rename(columns={'USA State':'Province_State'},inplace=True)
US_state_data['Province_State'] = US_state_data['Province_State'].replace({'District Of Columbia': 'District of Columbia'})
US_state_data.drop(US_state_data.index[0],axis=0,inplace=True)
US_state_semi_final = US_state_data.merge(df_merged1_US,on='Province_State',how='left')
US_state_semi_final.to_excel(r'US_state_semi_final.xlsx')

# China - Overall and statewise and curr prev

In [217]:
#Filter the global data to get the daily count for China
df_china= df_global[df_global.country == 'China']
df_china.reset_index(inplace=True)
df_china.drop(['index','region'],axis=1,inplace=True)
df_china.to_excel(r'raw_data_china.xlsx')
#******************************************************************
#CALCULATING CURR PREV DAY DIFF FOR CHINA
url17 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
df_rc = pd.read_csv(url1, index_col=0)
url18 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df_cc = pd.read_csv(url2, index_col=0)
url9 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df_dc = pd.read_csv(url3, index_col=0)
df_rc.reset_index(inplace=True)
df_cc.reset_index(inplace=True)
df_dc.reset_index(inplace=True)
#Renaming cols
df_cc.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
df_rc.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
df_dc.rename(columns={'Country/Region':'country','Province/State':'province_state'}, inplace=True)
#Taking only the data for china
df_china_r= df_rc[df_rc.country == 'China']
df_china_d= df_dc[df_dc.country == 'China']
df_china_c= df_cc[df_cc.country == 'China']
#confirmed
df_china_c.drop(columns=['Lat','Long'],inplace=True,axis=1)
df_china_c.rename(columns={df_china_c.columns[-1]:"current_day_confirmed",df_china_c.columns[-2]: "prev_day_confirmed"}, inplace = True)
df_china_c['newConfirmedcases'] = df_china_c['current_day_confirmed'] - df_china_c['prev_day_confirmed']
df_china_c_1 = df_china_c[['country','province_state','prev_day_confirmed','current_day_confirmed','newConfirmedcases']]
df_china_confirmed_grpd = df_china_c_1.groupby('province_state').sum()
#recovered
df_china_r.drop(columns=['Lat','Long'],inplace=True,axis=1)
df_china_r.rename(columns={df_china_r.columns[-1]:"current_day_recover",df_china_r.columns[-2]: "prev_day_recover"}, inplace = True)
df_china_r['newRecoveredcases'] = df_china_r['current_day_recover'] - df_china_r['prev_day_recover']
df_china_r_1 = df_china_r[['country','province_state','prev_day_recover','current_day_recover','newRecoveredcases']]
df_china_recover_grpd = df_china_r_1.groupby('province_state').sum()
#deaths
df_china_d.drop(columns=['Lat','Long'],inplace=True,axis=1)
df_china_d.rename(columns={df_china_d.columns[-1]:"current_day_death",df_china_d.columns[-2]: "prev_day_death"}, inplace = True)
df_china_d['newDeaths'] = df_china_d['current_day_death'] - df_china_d['prev_day_death']
df_china_d_1 = df_china_d[['country','province_state','prev_day_death','current_day_death','newDeaths']]
df_china_death_grpd = df_china_d_1.groupby('province_state').sum()
#Merge
df_merged_china = pd.concat([df_china_confirmed_grpd, df_china_recover_grpd], axis=1)
df_merged_china1 = pd.concat([df_merged_china,df_china_death_grpd],axis=1)
df_merged_china1['country'] = 'China'
df_merged_china1.reset_index(inplace=True)

# Brazil Overall 


In [218]:
state_labels = {'AC': 'Acre','AL': 'Alagoas','AM': 'Amazonas','AP': 'Amapá','BA': 'Bahia',
                'CE': 'Ceará','DF': 'Distrito Federal','ES': 'Espírito Santo','GO': 'Goiás','MA': 'Maranhão',
                'MG': 'Minas Gerais','MS': 'Mato Grosso do Sul','MT': 'Mato Grosso','PA': 'Pará','PB': 'Paraíba',
                'PE': 'Pernambuco','PI': 'Piauí','PR': 'Paraná','RJ': 'Rio de Janeiro','RN': 'Rio Grande do Norte',
                'RO': 'Rondônia','RR': 'Roraima','RS': 'Rio Grande do Sul','SC': 'Santa Catarinao','SE': 'Sergipe',
                'SP':'São Paulo','TO':'Tocantins'}
s = pd.Series(state_labels, name='State')
s1 = pd.DataFrame(s)
s2 = s1.reset_index()
s3 = s2.set_index('index')['State']

In [219]:
#brazil daily data statewise
url20 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
df_brazil = pd.read_csv(url20, index_col=0)
df_brazil.reset_index(inplace=True)
df_brazil.fillna(0, inplace=True)
df_brazil1=df_brazil[['date','country','state','deaths','recovered','totalCases']]
df_brazil1 = df_brazil1[~(df_brazil1.state.isin(["TOTAL"]))]
df_brazil1['state'] = df_brazil1['state'].replace(s3)
df_brazil1.to_excel('raw_data_Brazil.xlsx')
df_brazil2 =df_brazil1.copy()
#specific datasets
df_brazil1_confirmed = df_brazil2[['date','country','state','totalCases']]
df_brazil1_recover = df_brazil2[['date','country','state','recovered']]
df_brazil1_deaths = df_brazil2[['date','country','state','deaths']]
#inplace
df_brazil1_confirmed.reset_index(inplace=True)
df_brazil1_recover.reset_index(inplace=True)
df_brazil1_deaths.reset_index(inplace=True)
#drop
df_brazil1_deaths.drop('index',inplace=True,axis=1)
df_brazil1_recover.drop('index',inplace=True,axis=1)
df_brazil1_confirmed.drop('index',inplace=True,axis=1)

df_brazil1_deaths_pivot = df_brazil1_deaths.pivot_table(index=['country','state'],columns=['date'],values='deaths')
df_brazil1_deaths_pivot.reset_index(inplace=True)

df_brazil1_confirmed_pivot = df_brazil1_confirmed.pivot_table(index=['country','state'],columns=['date'],values='totalCases')
df_brazil1_confirmed_pivot.reset_index(inplace=True)

df_brazil1_recover_pivot = df_brazil1_recover.pivot_table(index=['country','state'],columns=['date'],values='recovered')
df_brazil1_recover_pivot.reset_index(inplace=True)

df_brazil1_confirmed_pivot.rename(columns={df_brazil1_deaths_pivot.columns[-1]:"current_day_confirmed",df_brazil1_confirmed_pivot.columns[-2]: "prev_day_confirmed" }, inplace = True)
df_brazil1_confirmed_pivot['prev_day_diff_confrimed'] = df_brazil1_confirmed_pivot['current_day_confirmed'] - df_brazil1_confirmed_pivot['prev_day_confirmed']
df_brazil1_confirmed_pivot1 = df_brazil1_confirmed_pivot[['country','state','prev_day_confirmed','current_day_confirmed','prev_day_diff_confrimed']]
df_brazil_confirmed_grpd = df_brazil1_confirmed_pivot1.groupby('state').sum()

df_brazil1_recover_pivot.rename(columns={df_brazil1_recover_pivot.columns[-1]:"current_day_recover",df_brazil1_recover_pivot.columns[-2]: "prev_day_recover" }, inplace = True)
df_brazil1_recover_pivot['prev_day_diff_recover'] = df_brazil1_recover_pivot['current_day_recover'] - df_brazil1_recover_pivot['prev_day_recover']
df_brazil1_recover_pivot1 = df_brazil1_recover_pivot[['country','state','prev_day_recover','current_day_recover','prev_day_diff_recover']]
df_brazil_recover_grpd = df_brazil1_recover_pivot1.groupby('state').sum()

df_brazil1_deaths_pivot.rename(columns={df_brazil1_deaths_pivot.columns[-1]:"current_day_death",df_brazil1_deaths_pivot.columns[-2]: "prev_day_death" }, inplace = True)
df_brazil1_deaths_pivot['prev_day_diff_death'] = df_brazil1_deaths_pivot['current_day_death'] - df_brazil1_deaths_pivot['prev_day_death']
df_brazil1_deaths_pivot1 = df_brazil1_deaths_pivot[['country','state','prev_day_death','current_day_death','prev_day_diff_death']]
df_brazil_deaths_grpd = df_brazil1_deaths_pivot1.groupby('state').sum()
#merge all tables 
df_merged1_Brazil = df_brazil_confirmed_grpd.merge(df_brazil_recover_grpd,on='state',how='left')
df_merged_Brazil = df_merged1_Brazil.merge(df_brazil_deaths_grpd,on='state',how='left')

In [220]:
#brazil statewise data load - aggregated
url21 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-total.csv'
df_brazil_agg = pd.read_csv(url21, index_col=0)
df_brazil_agg['state'] = df_brazil_agg['state'].replace(s3)
df_brazil_agg.reset_index(inplace=True)
df_brazil_agg.drop(df_brazil_agg.index[0],axis=0,inplace=True)
df_brazil_agg1 = df_brazil_agg.merge(df_merged_Brazil,on='state',how='left')